In [1]:
import numpy
import pandas as pd

#### Загрузка данных и обработка

In [24]:
data1 = pd.read_csv('.\hackathon\data1.csv', parse_dates=['date'])

data2 = pd.read_csv('.\hackathon\data2.csv', parse_dates=['date'])

data3 = pd.read_csv('.\hackathon\data3.csv', parse_dates=['date'])

data4 = pd.read_csv('.\hackathon\data4.csv', parse_dates=['date'])

data5 = pd.read_csv('.\hackathon\data5.csv', parse_dates=['date'])

In [25]:
print(data1.isna().sum())
print(data2.isna().sum())
print(data3.isna().sum())
print(data4.isna().sum())
print(data5.isna().sum())

value    0
date     0
dtype: int64
value    1
date     0
dtype: int64
value    6
date     0
dtype: int64
date     0
value    0
dtype: int64
date     0
value    0
dtype: int64


In [26]:
# Используем линейную интерполяцию для заполнения пропусков
data2['value'] = data2['value'].interpolate(method='linear')
data3['value'] = data3['value'].interpolate(method='linear')

#### Займёмся генерацией фичей для Датасета 1

In [27]:
data1.head()

,value,date
0,898.0,2022-01-01
1,3167.0,2022-01-02
2,3380.0,2022-01-03
3,3423.0,2022-01-04
4,3373.0,2022-01-05


In [28]:
from datetime import timedelta
import holidays

# Добавляем временные признаки

data1['date'] = pd.to_datetime(data1['date'])

data1['year'] = data1['date'].dt.year
data1['month'] = data1['date'].dt.month
data1['day'] = data1['date'].dt.day
data1['dayofweek'] = data1['date'].dt.dayofweek  # Понедельник=0, Воскресенье=6
data1['dayofyear'] = data1['date'].dt.dayofyear
data1['weekofyear'] = data1['date'].dt.isocalendar().week

# Добавляем информацию о праздниках (можно подстроить под страну)
holiday_list = holidays.Russia(years=data1['year'].iloc[0])  # Для России, можно изменить на другую страну
data1['is_holiday'] = data1['date'].isin(holiday_list)

data1.head()

,value,date,year,month,day,dayofweek,dayofyear,weekofyear,is_holiday
0,898.0,2022-01-01,2022,1,1,5,1,52,True
1,3167.0,2022-01-02,2022,1,2,6,2,52,True
2,3380.0,2022-01-03,2022,1,3,0,3,1,True
3,3423.0,2022-01-04,2022,1,4,1,4,1,True
4,3373.0,2022-01-05,2022,1,5,2,5,1,True


In [29]:
# Дополнительные признаки
data1['is_weekend'] = data1['dayofweek'].isin([5, 6])  # Суббота (5) и воскресенье (6)
data1['quarter'] = data1['date'].dt.quarter  # Квартал года

data1.head()


,value,date,year,month,day,dayofweek,dayofyear,weekofyear,is_holiday,is_weekend,quarter
0,898.0,2022-01-01,2022,1,1,5,1,52,True,True,1
1,3167.0,2022-01-02,2022,1,2,6,2,52,True,True,1
2,3380.0,2022-01-03,2022,1,3,0,3,1,True,False,1
3,3423.0,2022-01-04,2022,1,4,1,4,1,True,False,1
4,3373.0,2022-01-05,2022,1,5,2,5,1,True,False,1


In [30]:
# Добавим также лаговые признаки
# которые представляют собой значения временного ряда на предыдущих шагах
data1['lag_1'] = data1['value'].shift(1)  # Лаг 1
data1['lag_2'] = data1['value'].shift(2)  # Лаг 2
data1['lag_3'] = data1['value'].shift(3)  # Лаг 3

data1[['lag_1', 'lag_2', 'lag_3']] = data1[['lag_1', 'lag_2', 'lag_3']].fillna(0)

data1.head()

,value,date,year,month,day,dayofweek,dayofyear,weekofyear,is_holiday,is_weekend,quarter,lag_1,lag_2,lag_3
0,898.0,2022-01-01,2022,1,1,5,1,52,True,True,1,0.0,0.0,0.0
1,3167.0,2022-01-02,2022,1,2,6,2,52,True,True,1,898.0,0.0,0.0
2,3380.0,2022-01-03,2022,1,3,0,3,1,True,False,1,3167.0,898.0,0.0
3,3423.0,2022-01-04,2022,1,4,1,4,1,True,False,1,3380.0,3167.0,898.0
4,3373.0,2022-01-05,2022,1,5,2,5,1,True,False,1,3423.0,3380.0,3167.0


In [31]:
# Можно добавить агрегированные признаки на основе скользящих окон
# Например, среднее, медиану, минимум и максимум за последние 7 дней
data1['rolling_mean_7'] = data1['value'].rolling(window=7).mean()  # Скользящее среднее за 7 дней
data1['rolling_min_7'] = data1['value'].rolling(window=7).min()    # Скользящее минимальное значение за 7 дней
data1['rolling_max_7'] = data1['value'].rolling(window=7).max()    # Скользящее максимальное значение за 7 дней

data1[['rolling_mean_7', 'rolling_min_7', 'rolling_max_7']] = data1[['rolling_mean_7', 'rolling_min_7', 'rolling_max_7']].fillna(0)

data1.head(10)

,value,date,year,month,day,dayofweek,dayofyear,weekofyear,is_holiday,is_weekend,quarter,lag_1,lag_2,lag_3,rolling_mean_7,rolling_min_7,rolling_max_7
0,898.0,2022-01-01,2022,1,1,5,1,52,True,True,1,0.0,0.0,0.0,0.000000,0.0,0.0
1,3167.0,2022-01-02,2022,1,2,6,2,52,True,True,1,898.0,0.0,0.0,0.000000,0.0,0.0
2,3380.0,2022-01-03,2022,1,3,0,3,1,True,False,1,3167.0,898.0,0.0,0.000000,0.0,0.0
3,3423.0,2022-01-04,2022,1,4,1,4,1,True,False,1,3380.0,3167.0,898.0,0.000000,0.0,0.0
4,3373.0,2022-01-05,2022,1,5,2,5,1,True,False,1,3423.0,3380.0,3167.0,0.000000,0.0,0.0
5,3767.0,2022-01-06,2022,1,6,3,6,1,True,False,1,3373.0,3423.0,3380.0,0.000000,0.0,0.0
6,3511.0,2022-01-07,2022,1,7,4,7,1,True,False,1,3767.0,3373.0,3423.0,3074.142857,898.0,3767.0
7,3645.0,2022-01-08,2022,1,8,5,8,1,True,True,1,3511.0,3767.0,3373.0,3466.571429,3167.0,3767.0
8,3543.0,2022-01-09,2022,1,9,6,9,1,False,True,1,3645.0,3511.0,3767.0,3520.285714,3373.0,3767.0
9,2122.0,2022-01-10,2022,1,10,0,10,2,False,False,1,3543.0,3645.0,3511.0,3340.571429,2122.0,3767.0


In [44]:
data1_processed = data1.copy()

#### Создадим класс предобработки данных и применим его к остальным временным рядам

In [38]:
class TimeSeriesPreprocessor:
    def __init__(self, country='Russia', holiday_years=None, interpolate=True):
        self.country = country
        self.holiday_years = holiday_years
        self.interpolate = interpolate

    def add_time_features(self, df):
        # Добавление временных признаков
        # Преобразуем в формат datetime
        df['date'] = pd.to_datetime(df['date'])

        # Добавляем базовые признаки
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day
        df['dayofweek'] = df['date'].dt.dayofweek
        df['dayofyear'] = df['date'].dt.dayofyear
        df['weekofyear'] = df['date'].dt.isocalendar().week

        # Добавляем информацию о праздниках
        holiday_list = holidays.CountryHoliday(self.country, years=self.holiday_years if self.holiday_years else [df['year'].iloc[0]])
        df['is_holiday'] = df['date'].isin(holiday_list)

        # Дополнительные признаки
        df['is_weekend'] = df['dayofweek'].isin([5, 6])  # Суббота (5) и воскресенье (6)
        df['quarter'] = df['date'].dt.quarter  # Квартал года
        
        return df

    def add_lag_features(self, df, lags=[1, 2, 3]):
        for lag in lags:
            df[f'lag_{lag}'] = df['value'].shift(lag)  

        df[['lag_1', 'lag_2', 'lag_3']] = df[['lag_1', 'lag_2', 'lag_3']].fillna(0)

        return df
    
    def add_rolling_features(self, df, windows=[7]):
        for window in windows:
            df[f'rolling_mean_{window}'] = df['value'].rolling(window=window).mean()  # Скользящее среднее
            df[f'rolling_min_{window}'] = df['value'].rolling(window=window).min()  # Минимум
            df[f'rolling_max_{window}'] = df['value'].rolling(window=window).max()  # Максимум
        
        df[[f'rolling_mean_{window}' for window in windows]] = df[[f'rolling_mean_{window}' for window in windows]].fillna(0)
        df[[f'rolling_min_{window}' for window in windows]] = df[[f'rolling_min_{window}' for window in windows]].fillna(0)
        df[[f'rolling_max_{window}' for window in windows]] = df[[f'rolling_max_{window}' for window in windows]].fillna(0)

        return df
    
    def preprocess(self, df):
        if self.interpolate:
            df['value'] = df['value'].interpolate(method='linear')

        # Добавляем временные признаки
        df = self.add_time_features(df)

        # Добавляем лаговые признаки
        df = self.add_lag_features(df)

        # Добавляем скользящие признаки
        df = self.add_rolling_features(df)

        df = df.replace({True: 1, False: 0})

        return df


In [39]:
# Проверим работу препроцессора

preprocessor = TimeSeriesPreprocessor(country='Russia', holiday_years=[2022, 2023], interpolate=True)

data2_processed = preprocessor.preprocess(data2)

data2_processed.head(10)

,value,date,year,month,day,dayofweek,dayofyear,weekofyear,is_holiday,is_weekend,quarter,lag_1,lag_2,lag_3,rolling_mean_7,rolling_min_7,rolling_max_7
0,332.0,2022-01-01,2022,1,1,5,1,52,1,1,1,0.0,0.0,0.0,0.000000,0.0,0.0
1,1011.0,2022-01-02,2022,1,2,6,2,52,1,1,1,332.0,0.0,0.0,0.000000,0.0,0.0
2,1102.0,2022-01-03,2022,1,3,0,3,1,1,0,1,1011.0,332.0,0.0,0.000000,0.0,0.0
3,1065.0,2022-01-04,2022,1,4,1,4,1,1,0,1,1102.0,1011.0,332.0,0.000000,0.0,0.0
4,819.0,2022-01-05,2022,1,5,2,5,1,1,0,1,1065.0,1102.0,1011.0,0.000000,0.0,0.0
5,721.0,2022-01-06,2022,1,6,3,6,1,1,0,1,819.0,1065.0,1102.0,0.000000,0.0,0.0
6,759.0,2022-01-07,2022,1,7,4,7,1,1,0,1,721.0,819.0,1065.0,829.857143,332.0,1102.0
7,878.0,2022-01-08,2022,1,8,5,8,1,1,1,1,759.0,721.0,819.0,907.857143,721.0,1102.0
8,598.0,2022-01-09,2022,1,9,6,9,1,0,1,1,878.0,759.0,721.0,848.857143,598.0,1102.0
9,154.0,2022-01-10,2022,1,10,0,10,2,0,0,1,598.0,878.0,759.0,713.428571,154.0,1065.0


In [40]:
data3_processed = preprocessor.preprocess(data3)

data4_processed = preprocessor.preprocess(data4)

data5_processed = preprocessor.preprocess(data5)

#### Выполним проверку на выбросы и аномалии

In [42]:
from scipy import stats
import numpy as np

# Проверяем выбросы для одного из временных рядов
z_scores = np.abs(stats.zscore(data1['value']))
outliers = np.where(z_scores > 3)  # Значение 3 - это порог для выбросов
print("Indices of outliers:", outliers)

Indices of outliers: (array([362, 363, 364, 700, 721, 726, 727, 728, 729, 730], dtype=int64),)


#### Декомпозиция временных рядов

In [50]:
# Добавляем столбец 'segment' в каждый DataFrame
data1_processed['segment'] = 'store_1'
data2_processed['segment'] = 'store_2'
data3_processed['segment'] = 'store_3'
data4_processed['segment'] = 'store_4'
data5_processed['segment'] = 'store_5'

all_data = pd.concat([data1_processed, data2_processed, data3_processed, data4_processed, data5_processed], axis=0)

all_data = all_data.sort_values(by=['segment', 'date'])

# Проверим результат
all_data.head()

,value,date,year,month,day,dayofweek,dayofyear,weekofyear,is_holiday,is_weekend,quarter,lag_1,lag_2,lag_3,rolling_mean_7,rolling_min_7,rolling_max_7,segment
0,898.0,2022-01-01,2022,1,1,5,1,52,True,True,1,0.0,0.0,0.0,0.0,0.0,0.0,store_1
1,3167.0,2022-01-02,2022,1,2,6,2,52,True,True,1,898.0,0.0,0.0,0.0,0.0,0.0,store_1
2,3380.0,2022-01-03,2022,1,3,0,3,1,True,False,1,3167.0,898.0,0.0,0.0,0.0,0.0,store_1
3,3423.0,2022-01-04,2022,1,4,1,4,1,True,False,1,3380.0,3167.0,898.0,0.0,0.0,0.0,store_1
4,3373.0,2022-01-05,2022,1,5,2,5,1,True,False,1,3423.0,3380.0,3167.0,0.0,0.0,0.0,store_1


In [61]:
from statsmodels.tsa.seasonal import STL

decomposed_data = []

# Применим декомпозицию для каждого сегмента (магазина)
for segment in all_data['segment'].unique():
    store_data = all_data[all_data['segment'] == segment]
    
    # Преобразуем столбец 'date' в DatetimeIndex, если это ещё не сделано
    store_data['date'] = pd.to_datetime(store_data['date'])
    
    # Устанавливаем индекс по дате
    store_data.set_index('date', inplace=True)
    
    # Проверяем количество данных для этого сегмента
    n_obs = len(store_data)
    if n_obs >= 7:
        # Декомпозиция временного ряда с использованием STL и периодом 365 (для годовых данных)
        stl = STL(store_data['value'], seasonal=365)  # Частота 365 дней
        decomposition = stl.fit()
        
        # Добавляем компоненты к данным
        store_data['trend'] = decomposition.trend
        store_data['seasonal'] = decomposition.seasonal
        store_data['residual'] = decomposition.resid
    else:
        # Если данных недостаточно, можно просто пропустить этот сегмент или использовать другую стратегию
        store_data['trend'] = None
        store_data['seasonal'] = None
        store_data['residual'] = None
    
    decomposed_data.append(store_data)

# Объединяем декомпозированные данные
final_data = pd.concat(decomposed_data, axis=0)

# Проверяем результат
final_data.head()


C:\Users\denis\AppData\Local\Temp\ipykernel_12212\2370399319.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_data['date'] = pd.to_datetime(store_data['date'])
C:\Users\denis\AppData\Local\Temp\ipykernel_12212\2370399319.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_data['trend'] = decomposition.trend
C:\Users\denis\AppData\Local\Temp\ipykernel_12212\2370399319.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,value,year,month,day,dayofweek,dayofyear,weekofyear,is_holiday,is_weekend,quarter,lag_1,lag_2,lag_3,rolling_mean_7,rolling_min_7,rolling_max_7,segment,trend,seasonal,residual
date,,,,,,,,,,,,,,,,,,,,
2022-01-01,898.0,2022,1,1,5,1,52,True,True,1,0.0,0.0,0.0,0.0,0.0,0.0,store_1,1830.033608,1009.162777,-1941.196385
2022-01-02,3167.0,2022,1,2,6,2,52,True,True,1,898.0,0.0,0.0,0.0,0.0,0.0,store_1,2258.442208,558.645832,349.911960
2022-01-03,3380.0,2022,1,3,0,3,1,True,False,1,3167.0,898.0,0.0,0.0,0.0,0.0,store_1,2639.085928,-483.151882,1224.065954
2022-01-04,3423.0,2022,1,4,1,4,1,True,False,1,3380.0,3167.0,898.0,0.0,0.0,0.0,store_1,2990.603480,-444.975375,877.371894
2022-01-05,3373.0,2022,1,5,2,5,1,True,False,1,3423.0,3380.0,3167.0,0.0,0.0,0.0,store_1,3321.726430,-485.792853,537.066424


In [63]:
# Применяем One-Hot Encoding для признака 'segment'
data_encoded = pd.get_dummies(final_data, columns=['segment'], drop_first=True)

data_encoded = data_encoded.replace({True: 1, False: 0})

# Проверяем результат
data_encoded.head()

,value,year,month,day,dayofweek,dayofyear,weekofyear,is_holiday,is_weekend,quarter,...,rolling_mean_7,rolling_min_7,rolling_max_7,trend,seasonal,residual,segment_store_2,segment_store_3,segment_store_4,segment_store_5
date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,898.0,2022,1,1,5,1,52,1,1,1,...,0.0,0.0,0.0,1830.033608,1009.162777,-1941.196385,0,0,0,0
2022-01-02,3167.0,2022,1,2,6,2,52,1,1,1,...,0.0,0.0,0.0,2258.442208,558.645832,349.911960,0,0,0,0
2022-01-03,3380.0,2022,1,3,0,3,1,1,0,1,...,0.0,0.0,0.0,2639.085928,-483.151882,1224.065954,0,0,0,0
2022-01-04,3423.0,2022,1,4,1,4,1,1,0,1,...,0.0,0.0,0.0,2990.603480,-444.975375,877.371894,0,0,0,0
2022-01-05,3373.0,2022,1,5,2,5,1,1,0,1,...,0.0,0.0,0.0,3321.726430,-485.792853,537.066424,0,0,0,0


#### Приступим к обучению моделей

In [71]:
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Целевая переменная
y = data_encoded['value']

X = data_encoded.drop(columns=['value'], axis=1)

# Преобразуем все столбцы в int или float (если это необходимо)
X = X.astype({'weekofyear': 'int32',  # Преобразуем столбец 'weekofyear' в int32
              'year': 'int32',         # Пример, если есть столбец с типом 'UInt32'
              'month': 'int32',        # Преобразуем другие столбцы, если необходимо
              'day': 'int32',          # Преобразуем другие столбцы, если необходимо
              'dayofweek': 'int32',    # Преобразуем другие столбцы, если необходимо
              'dayofyear': 'int32'})   # Преобразуем другие столбцы, если необходимо

tscv = TimeSeriesSplit(n_splits=5)  

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, max_depth=6)

rmse_scores = []

# Пройдем по каждому фолду
for train_index, test_index in tscv.split(X):
    # Разбиваем на тренировочную и тестовую выборки
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Обучаем модель
    model.fit(X_train, y_train)
    
    # Прогнозируем на тестовой выборке
    y_pred = model.predict(X_test)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_scores.append(rmse)

# Выводим RMSE для каждого фолда
print(f"RMSE for each fold: {rmse_scores}")
print(f"Average RMSE: {np.mean(rmse_scores)}")

RMSE for each fold: [1500.468490126271, 141.75671475501383, 34.89851787392157, 240.79448639652495, 63.68388529901557]
Average RMSE: 396.32041889014937


In [73]:
import pickle

# Сохраняем модель с помощью pickle
model_path = "xgboost_model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

In [69]:
from sklearn.model_selection import GridSearchCV

# Инициализация модели XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, max_depth=6)

# Определяем параметры для поиска
param_grid = {
    'learning_rate': [0.05, 0.1, 0.2],  # Увеличиваем learning_rate
    'max_depth': [5, 6, 7],              # Уменьшаем max_depth
    'n_estimators': [100, 150],          # Уменьшаем количество деревьев
    'min_child_weight': [1, 3, 5],        # Регуляризация
    'gamma': [0, 0.1],                   # Регуляризация
}

# Инициализация GridSearchCV с TimeSeriesSplit и моделью XGBoost
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=tscv, n_jobs=-1, verbose=1)

# Запуск поиска
grid_search.fit(X, y)

# Выводим лучшие параметры и лучший результат
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best RMSE: {np.sqrt(-grid_search.best_score_)}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 150}
Best RMSE: 528.8327404775234
